**Upload Kaggle.json and stopwords.txt**

In [20]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 71 bytes


In [21]:
#"""
!kaggle competitions download -c nbme-score-clinical-patient-notes
!unzip -o /content/nbme-score-clinical-patient-notes.zip
#"""

nbme-score-clinical-patient-notes.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/nbme-score-clinical-patient-notes.zip
  inflating: features.csv            
  inflating: patient_notes.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [22]:
import pandas as pd
import numpy as np
import ast
import itertools
from scipy import sparse
from string import punctuation
from sklearn.tree import DecisionTreeClassifier

In [23]:
stopwords = ["x", "your", "yours", "yourself", "yourselves", "you", "yond", "yonder", "yon", "ye", "yet", "z", "zillion", "j", "u", "umpteen", "usually", "us", "username", "uponed", "upons", "uponing", "upon", "ups", "upping", "upped", "up", "unto", "until", "unless", "unlike", "unliker", "unlikest", "under", "underneath", "use", "used", "usedest", "r", "rath", "rather", "rathest", "rathe", "re", "relate", "related", "relatively", "regarding", "really", "res", "respecting", "respectively", "q", "quite", "que", "qua", "n", "neither", "neaths", "neath", "nethe", "nethermost", "necessary", "necessariest", "necessarier", "never", "nevertheless", "nigh", "nighest", "nigher", "nine", "noone", "nobody", "nobodies", "nowhere", "nowheres", "no", "noes", "nor", "nos", "no-one", "none", "not", "notwithstanding", "nothings", "nothing", "nathless", "natheless", "t", "ten", "tills", "till", "tilled", "tilling", "to", "towards", "toward", "towardest", "towarder", "together", "too", "thy", "thyself", "thus", "than", "that", "those", "thou", "though", "thous", "thouses", "thoroughest", "thorougher", "thorough", "thoroughly", "thru", "thruer", "thruest", "thro", "through", "throughout", "throughest", "througher", "thine", "this", "thises", "they", "thee", "the", "then", "thence", "thenest", "thener", "them", "themselves", "these", "therer", "there", "thereby", "therest", "thereafter", "therein", "thereupon", "therefore", "their", "theirs", "thing", "things", "three", "two", "oh", "owt", "owning", "owned", "own", "owns", "others", "other", "otherwise", "otherwisest", "otherwiser", "of", "often", "oftener", "oftenest", "off", "offs", "offest", "one", "ought", "oughts", "our", "ours", "ourselves", "ourself", "out", "outest", "outed", "outwith", "outs", "outside", "over", "overallest", "overaller", "overalls", "overall", "overs", "or", "orer", "orest", "on", "oneself", "onest", "ons", "onto", "a", "atween", "at", "athwart", "atop", "afore", "afterward", "afterwards", "after", "afterest", "afterer", "ain", "an", "any", "anything", "anybody", "anyone", "anyhow", "anywhere", "anent", "anear", "and", "andor", "another", "around", "ares", "are", "aest", "aer", "against", "again", "accordingly", "abaft", "abafter", "abaftest", "abovest", "above", "abover", "abouter", "aboutest", "about", "aid", "amidst", "amid", "among", "amongst", "apartest", "aparter", "apart", "appeared", "appears", "appear", "appearing", "appropriating", "appropriate", "appropriatest", "appropriates", "appropriater", "appropriated", "already", "always", "also", "along", "alongside", "although", "almost", "all", "allest", "aller", "allyou", "alls", "albeit", "awfully", "as", "aside", "asides", "aslant", "ases", "astrider", "astride", "astridest", "astraddlest", "astraddler", "astraddle", "availablest", "availabler", "available", "aughts", "aught", "vs", "v", "variousest", "variouser", "various", "via", "vis-a-vis", "vis-a-viser", "vis-a-visest", "viz", "very", "veriest", "verier", "versus", "k", "g", "go", "gone", "good", "got", "gotta", "gotten", "get", "gets", "getting", "b", "by", "byandby", "by-and-by", "bist", "both", "but", "buts", "be", "beyond", "because", "became", "becomes", "become", "becoming", "becomings", "becominger", "becomingest", "behind", "behinds", "before", "beforehand", "beforehandest", "beforehander", "bettered", "betters", "better", "bettering", "betwixt", "between", "beneath", "been", "below", "besides", "beside", "my", "myself", "mucher", "muchest", "much", "must", "musts", "musths", "musth", "main", "make", "mayest", "many", "mauger", "maugre", "me", "meanwhiles", "meanwhile", "mostly", "most", "moreover", "more", "might", "mights", "midst", "midsts", "h", "huh", "humph", "he", "hers", "herself", "her", "hereby", "herein", "hereafters", "hereafter", "hereupon", "hence", "hadst", "had", "having", "haves", "have", "has", "hast", "hardly", "hae", "hath", "him", "himself", "hither", "hitherest", "hitherer", "his", "how-do-you-do", "however", "how", "howbeit", "howdoyoudo", "hoos", "hoo", "w", "woulded", "woulding", "would", "woulds", "was", "wast", "we", "wert", "were", "with", "withal", "without", "within", "why", "what", "whatever", "whateverer", "whateverest", "whatsoeverer", "whatsoeverest", "whatsoever", "whence", "whencesoever", "whenever", "whensoever", "when", "whenas", "whether", "wheen", "whereto", "whereupon", "wherever", "whereon", "whereof", "where", "whereby", "wherewithal", "wherewith", "whereinto", "wherein", "whereafter", "whereas", "wheresoever", "wherefrom", "which", "whichever", "whichsoever", "whilst", "while", "whiles", "whithersoever", "whither", "whoever", "whosoever", "whoso", "whose", "whomever", "s", "syne", "syn", "shalling", "shall", "shalled", "shalls", "shoulding", "should", "shoulded", "shoulds", "she", "sayyid", "sayid", "said", "saider", "saidest", "same", "samest", "sames", "samer", "saved", "sans", "sanses", "sanserifs", "sanserif", "so", "soer", "soest", "sobeit", "someone", "somebody", "somehow", "some", "somewhere", "somewhat", "something", "sometimest", "sometimes", "sometimer", "sometime", "several", "severaler", "severalest", "serious", "seriousest", "seriouser", "senza", "send", "sent", "seem", "seems", "seemed", "seemingest", "seeminger", "seemings", "seven", "summat", "sups", "sup", "supping", "supped", "such", "since", "sine", "sines", "sith", "six", "stop", "stopped", "p", "plaintiff", "plenty", "plenties", "please", "pleased", "pleases", "per", "perhaps", "particulars", "particularly", "particular", "particularest", "particularer", "pro", "providing", "provides", "provided", "provide", "probably", "l", "layabout", "layabouts", "latter", "latterest", "latterer", "latterly", "latters", "lots", "lotting", "lotted", "lot", "lest", "less", "ie", "ifs", "if", "i", "info", "information", "itself", "its", "it", "is", "idem", "idemer", "idemest", "immediate", "immediately", "immediatest", "immediater", "in", "inwards", "inwardest", "inwarder", "inward", "inasmuch", "into", "instead", "insofar", "indicates", "indicated", "indicate", "indicating", "indeed", "inc", "f", "fact", "facts", "fs", "figupon", "figupons", "figuponing", "figuponed", "few", "fewer", "fewest", "frae", "from", "failing", "failings", "five", "furthers", "furtherer", "furthered", "furtherest", "further", "furthering", "furthermore", "fourscore", "followthrough", "for", "forwhy", "fornenst", "formerly", "former", "formerer", "formerest", "formers", "forbye", "forby", "fore", "forever", "forer", "fores", "four", "d", "ddays", "dday", "do", "doing", "doings", "doe", "does", "doth", "downwarder", "downwardest", "downward", "downwards", "downs", "done", "doner", "dones", "donest", "dos", "dost", "did", "differentest", "differenter", "different", "describing", "describe", "describes", "described", "despiting", "despites", "despited", "despite", "during", "c", "cum", "circa", "chez", "cer", "certain", "certainest", "certainer", "cest", "canst", "cannot", "cant", "cants", "canting", "cantest", "canted", "co", "could", "couldst", "comeon", "comeons", "come-ons", "come-on", "concerning", "concerninger", "concerningest", "consequently", "considering", "e", "eg", "eight", "either", "even", "evens", "evenser", "evensest", "evened", "evenest", "ever", "everyone", "everything", "everybody", "everywhere", "every", "ere", "each", "et", "etc", "elsewhere", "else", "ex", "excepted", "excepts", "except", "excepting", "exes", "enough",]

In [24]:
def tokenizeNBME(txt):
  lst = []
  st = 0
  for i in range(len(txt)):
    if txt[i] in punctuation+' \r\n':
      lst.append(txt[st:i])
      lst.append(txt[i])
      i+=1
      st = i
  lst.append(txt[st:])
  return "#*#".join(lst)
def getValidWords(txt):
  lst = []
  st = 0
  for i in range(len(txt)):
    if txt[i] in punctuation+' \r\n':
      if txt[st:i]!='':
        lst.append(txt[st:i])
      i+=1
      st = i
  if txt[st:i]!='':
    lst.append(txt[st:])
  return list(set(lst))
def countWordsInDf(lst):
  wordDict = {}
  for word in lst:
    if word in wordDict:
      wordDict[word] = wordDict[word] + 1
    else:
      wordDict[word] = 1
  return wordDict
def flattenArray(array):
  x = []
  for i in array:
    for j in i:
      x.append(j)
  return x
def getExpanded(x):
  ex_list = []
  for val in x:
    lst = val.split(";")
    for lst1 in lst:
      lst2 = lst1.split(" ")
      ex_list.extend(list(range(int(lst2[0]), int(lst2[1]))))
  return ex_list
def getEleInLoc(x,ele):
  ex_list = getExpanded(x)
  return 1 if int(ele) in ex_list else 0
def evaluateEncode(pn_history, annotation, location):
  lst = [['startOfSequence', 1, 0, len(annotation), 0]]
  #lst=[]
  len1 = 0
  for i in range(len(pn_history)):
    if pn_history[i] in annotation:
      #word, index/pos, order/dist, count, loc exist?
      lst.append([pn_history[i], len1, i,  annotation[pn_history[i]], getEleInLoc(location, len1)])
      len1 += len(pn_history[i])
    else:
      len1 += len(pn_history[i])
  if len(lst) ==1:
    pnj = "".join(pn_history)
    values_to_look = (sorted(annotation, key = annotation.get, reverse=True))
    for j in range(len(values_to_look)):
      if values_to_look[j] in pnj:
        lst.append([values_to_look[j], pnj.index(values_to_look[j]), 2, annotation[values_to_look[j]], getEleInLoc(location, pnj.index(values_to_look[j]))])
        break
  if len(lst) ==1:
    lst.append([(sorted(annotation, key = annotation.get, reverse=True))[0], 0, 2, 1, 0])
  lst.append(['endOfSequence', 1, len(pn_history)+1, len(annotation), 0])
  return np.array(lst)
def getConfusionDataframe(annotation_flt, ann_vld):
  df_conf = pd.DataFrame(np.zeros((len(ann_vld),len(ann_vld)), dtype=np.int32))
  df_conf.columns = ann_vld
  df_conf.index = ann_vld
  for line in annotation_flt:
    words = getValidWords(line)
    if len(words)==1:
      #print(words)
      df_conf[words[0]][words[0]] = df_conf[words[0]][words[0]] + 1
      continue
    for combn in list(itertools.combinations(words, 2)):
      df_conf[combn[0]][combn[1]] = df_conf[combn[0]][combn[1]] + 1
      df_conf[combn[1]][combn[0]] = df_conf[combn[1]][combn[0]] + 1
  return df_conf
def getAOW(x, y, conf_df):
  try:
    return conf_df[x][y]
  except:
    return 1
def prepareData_v1(seq, conf_df):
  lst=[]
  for i in range(1,len(seq)-1):
    #word, dist from prev, dist from next, count_current, count_prev, count_next, attchment_prev, attchment_next, output
    lst.append([seq[i][0], (int(seq[i][2]) - int(seq[i-1][2])) , (int(seq[i+1][2]) - int(seq[i][2])), int(seq[i][3]), int(seq[i-1][3]), int(seq[i+1][3]), getAOW(seq[i][0],seq[i-1][0], conf_df), getAOW(seq[i][0],seq[i+1][0], conf_df), int(seq[i][4]) ])
  return lst
hp_gap=8
def predSlices(seq, gap=hp_gap):
  lst = []
  s0 = seq[0]
  for i in range(1, len(seq)):
    if (int(seq[i][2]) - int(seq[i-1][2])) > gap:
      lst.append([int(s0[1]), int(seq[i-1][1]) + len(seq[i-1][0])])
      s0 = seq[i]
  lst.append([int(s0[1]), int(seq[-1][1]) + len(seq[-1][0])])
  return lst

curr_ptr = 0
def get_next_sequence_pred(seq):
  global curr_ptr
  nxt_seq = seq[1:-1][(y_pred[curr_ptr:curr_ptr+len(seq[1:-1])]).astype('bool')]
  curr_ptr += len(seq[1:-1])
  if len(nxt_seq) == 0:
    return seq[1:-1]
  return nxt_seq
def getXY(data, wordDict):
  lstx=[]
  lsty=[]
  for x in data:
    arr = np.zeros(len(wordDict), dtype=np.int32)
    arr[list(wordDict.keys()).index(x[0])] = 1
    lstx.append(np.hstack([np.array(x[1:-1]), arr]))
    lsty.append(np.array(x[-1]))
  return [lstx, lsty]
def getF1Score(y_true, y_pred):
  return len(set(y_true).intersection(set(y_pred))) / (len(set(y_true).intersection(set(y_pred))) + 0.5*len(set(y_true).symmetric_difference(set(y_pred))))
#########################Pred Methods###########################
def evaluateEncode_v2(pn_history, annotation):
  lst = [['startOfSequence', 1, 0, len(annotation)]]
  #lst=[]
  len1 = 0
  for i in range(len(pn_history)):
    if pn_history[i] in annotation:
      #word, index/pos, order/dist, count, loc exist?
      lst.append([pn_history[i], len1, i,  annotation[pn_history[i]]])
      len1 += len(pn_history[i])
    else:
      len1 += len(pn_history[i])
  if len(lst) ==1:
    pnj = "".join(pn_history)
    values_to_look = (sorted(annotation, key = annotation.get, reverse=True))
    for j in range(len(values_to_look)):
      if values_to_look[j] in pnj:
        lst.append([values_to_look[j], pnj.index(values_to_look[j]), 2, annotation[values_to_look[j]]])
        break
  if len(lst) ==1:
    lst.append([(sorted(annotation, key = annotation.get, reverse=True))[0], 0, 2, 1])
  lst.append(['endOfSequence', 1, len(pn_history)+1, len(annotation)])
  return np.array(lst)
def prepareData_v2(seq, conf_df):
  lst=[]
  for i in range(1,len(seq)-1):
    #word, dist from prev, dist from next, count_current, count_prev, count_next, attchment_prev, attchment_next, output
    lst.append([seq[i][0], (int(seq[i][2]) - int(seq[i-1][2])) , (int(seq[i+1][2]) - int(seq[i][2])), int(seq[i][3]), int(seq[i-1][3]), int(seq[i+1][3]), getAOW(seq[i][0],seq[i-1][0], conf_df), getAOW(seq[i][0],seq[i+1][0], conf_df) ])
  return lst
def getX(data, wordDict):
  lstx=[]
  for x in data:
    arr = np.zeros(len(wordDict), dtype=np.int32)
    arr[list(wordDict.keys()).index(x[0])] = 1
    lstx.append(np.hstack([np.array(x[1:]), arr]))
  return lstx
def get_next_sequence_pred_v2(seq, y_pred):
  nxt_seq = seq[1:-1][(y_pred[0:len(seq[1:-1])]).astype('bool')]
  if len(nxt_seq) == 0:
    return seq[1:-1]
  return nxt_seq
def get_x_train_v2(ftr_enc, conf_df, annotation_grp_cnt):
  prep_v2 = prepareData_v2(ftr_enc, conf_df)
  XY = getX(prep_v2, annotation_grp_cnt)
  x_train = []
  for xy_1 in XY:
    for x_1 in xy_1:
      x_train.append(x_1)
  return XY

In [6]:
##################################################################################

In [25]:
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("patient_notes.csv")
df3 = pd.read_csv("features.csv")
df4 = pd.read_csv("test.csv")
df_test = pd.merge(df4, df2, how='left', on = 'pn_num').merge(df3, how = 'left', on = 'feature_num')
df = pd.merge(df1, df2, how='left', on = 'pn_num').merge(df3, how = 'left', on = 'feature_num') ; del df1, df2, df3,df4
df.pn_history = df.pn_history.str.lower()
df.annotation = df.annotation.str.lower()
df.annotation = df.annotation.apply(lambda x: ast.literal_eval(x))
df.location = df.location.apply(lambda x: ast.literal_eval(x))
df.drop(columns=['id','case_num_x','pn_num','feature_num','case_num_y', 'case_num'], inplace=True)
df.to_csv('df.csv', index=False)

In [8]:
df_val = df[df.annotation.apply(lambda x: len(x)!=0)].reset_index(drop=True)
df_val['annotation_unp'] = df_val.annotation.apply(lambda x: " ".join(x))
df_val['annotationNBME'] = df_val.annotation_unp.apply(tokenizeNBME)
df_val['pn_historyNBME'] = df_val.pn_history.apply(tokenizeNBME)

In [9]:
ftr2 = pd.DataFrame(pd.DataFrame(df_val[['annotationNBME','feature_text']].groupby('feature_text')['annotationNBME'].apply(lambda x : "#*#".join(x))).annotationNBME.apply(lambda x: [x for x in (x.split("#*#")) if (x not in punctuation+" " and x not in stopwords and not x.isdigit())]))
#ftr2 = pd.DataFrame(pd.DataFrame(df_val[['annotationNBME','feature_text']].groupby('feature_text')['annotationNBME'].apply(lambda x : "#*#".join(x))).annotationNBME.apply(lambda x: [x for x in (x.split("#*#")) if (x not in punctuation+" ")]))
ftr2['feature_text'] = ftr2.index
ftr2.columns = ['annotation_grp_cnt','feature_text_grp_cnt']
ftr2['annotation_grp_cnt'] = ftr2.annotation_grp_cnt.apply(countWordsInDf)

In [10]:
ftr3 = pd.DataFrame(df_val[['annotation','feature_text']].groupby('feature_text')['annotation'].apply(lambda x: list(x)))
ftr3['annotation_flt'] = ftr3.annotation.apply(lambda x: flattenArray(x))
ftr3['annotation_vld'] = ftr3.annotation_flt.apply(lambda x: getValidWords(" ".join(x)))
ftr3['conf_df'] = ftr3.apply(lambda x: getConfusionDataframe(x.annotation_flt, x.annotation_vld), axis=1)
ftr3['feature_text'] = ftr3.index
ftr3.columns = ['annotation', 'annotation_flt', 'annotation_vld', 'conf_df','feature_text_conf']

In [11]:
df_val = pd.merge(df_val, ftr2,  left_on='feature_text', right_on='feature_text_grp_cnt').drop(columns=['feature_text_grp_cnt'])
df_val = pd.merge(df_val, ftr3[ftr3.columns[1:]],  left_on='feature_text', right_on='feature_text_conf').drop(columns=['feature_text_conf'])
df_val

,annotation,location,pn_history,feature_text,annotation_unp,annotationNBME,pn_historyNBME,annotation_grp_cnt,annotation_flt,annotation_vld,conf_df
0,[dad with recent heart attcak],[696 724],hpi: 17yo m presents with palpitations. patien...,Family-history-of-MI-OR-Family-history-of-myoc...,dad with recent heart attcak,dad#*# #*#with#*# #*#recent#*# #*#heart#*# #*#...,hpi#*#:#*##*# #*#17yo#*# #*#m#*# #*#presents#*...,"{'dad': 26, 'recent': 4, 'heart': 35, 'attcak'...","[dad with recent heart attcak, father: heart a...","[suffer, mi, father, acute, 1, attcak, in, the...",suffer mi father acute 1 attc...
1,[father: heart attack],[824 844],mr. cleveland is a 17yo m who was consented by...,Family-history-of-MI-OR-Family-history-of-myoc...,father: heart attack,father#*#:#*##*# #*#heart#*# #*#attack,mr#*#.#*##*# #*#cleveland#*# #*#is#*# #*#a#*# ...,"{'dad': 26, 'recent': 4, 'heart': 35, 'attcak'...","[dad with recent heart attcak, father: heart a...","[suffer, mi, father, acute, 1, attcak, in, the...",suffer mi father acute 1 attc...
2,[father mi],[622 631],17 yo m w/ no cardiac or arrhythmia pmh presen...,Family-history-of-MI-OR-Family-history-of-myoc...,father mi,father#*# #*#mi,17#*# #*#yo#*# #*#m#*# #*#w#*#/#*##*# #*#no#*#...,"{'dad': 26, 'recent': 4, 'heart': 35, 'attcak'...","[dad with recent heart attcak, father: heart a...","[suffer, mi, father, acute, 1, attcak, in, the...",suffer mi father acute 1 attc...
3,[dad-mi],[735 741],hpi: dillon cleveland is an otherwise healthy ...,Family-history-of-MI-OR-Family-history-of-myoc...,dad-mi,dad#*#-#*#mi,hpi#*#:#*##*# #*#dillon#*# #*#cleveland#*# #*#...,"{'dad': 26, 'recent': 4, 'heart': 35, 'attcak'...","[dad with recent heart attcak, father: heart a...","[suffer, mi, father, acute, 1, attcak, in, the...",suffer mi father acute 1 attc...
4,[father had acute mi],[601 620],"17 y/o m , dillon cleveland comes with c/o of ...",Family-history-of-MI-OR-Family-history-of-myoc...,father had acute mi,father#*# #*#had#*# #*#acute#*# #*#mi,"17#*# #*#y#*#/#*#o#*# #*#m#*# #*##*#,#*##*# #*...","{'dad': 26, 'recent': 4, 'heart': 35, 'attcak'...","[dad with recent heart attcak, father: heart a...","[suffer, mi, father, acute, 1, attcak, in, the...",suffer mi father acute 1 attc...
...,...,...,...,...,...,...,...,...,...,...,...
9896,[neck pain],[218 227],20 year odl female c/o headaches x few hrs. he...,Neck-pain,neck pain,neck#*# #*#pain,20#*# #*#year#*# #*#odl#*# #*#female#*# #*#c#*...,"{'neck': 39, 'stiffness': 16, 'pain': 17, 'tig...","[neck stiffness, neck pain, neck stiffness, ne...","[ve, tightness, rigidity, in, nuchal, pain, he...",ve tightness rigidity in nu...
9897,[neck pain],[158 167],pt is 20 yo f w headache since yesterday morni...,Neck-pain,neck pain,neck#*# #*#pain,pt#*# #*#is#*# #*#20#*# #*#yo#*# #*#f#*# #*#w#...,"{'neck': 39, 'stiffness': 16, 'pain': 17, 'tig...","[neck stiffness, neck pain, neck stiffness, ne...","[ve, tightness, rigidity, in, nuchal, pain, he...",ve tightness rigidity in nu...
9898,[neck stiffness],[338 352],stephanie madden is a 20 year old woman compla...,Neck-pain,neck stiffness,neck#*# #*#stiffness,stephanie#*# #*#madden#*# #*#is#*# #*#a#*# #*#...,"{'neck': 39, 'stiffness': 16, 'pain': 17, 'tig...","[neck stiffness, neck pain, neck stiffness, ne...","[ve, tightness, rigidity, in, nuchal, pain, he...",ve tightness rigidity in nu...
9899,[unsure of meningitis shot],[512 521;544 559],stephanie madden is a 20 year old female who p...,Meningococcal-vaccine-status-unknown,unsure of meningitis shot,unsure#*# #*#of#*# #*#meningitis#*# #*#shot,stephanie#*# #*#madden#*# #*#is#*# #*#a#*# #*#...,"{'unsure': 1, 'meningitis': 1, 'shot': 1, 'men...","[unsure of meningitis shot, did not get mening...","[shot, unsure, get, vaccinations, of, not, men...",shot unsure get vaccinations...


In [12]:
#df_val.to_csv("df_val.csv", index=False)
df_val.groupby('feature_text').count().location.sort_values().tail(20)

feature_text
Episodes-of-heart-racing               95
Vomiting                               95
Prior-episodes-of-diarrhea             96
Son-died-3-weeks-ago                   97
44-year                                98
26-year                                98
67-year                                99
Chest-pain                             99
Difficulty-with-sleep                  99
heart-pounding-OR-heart-racing         99
heavy-periods-OR-irregular-periods     99
Irregular-menses                      100
anxious-OR-nervous                    100
45-year                               100
Nausea                                157
17-year                               195
20-year                               197
35-year                               198
Male                                  286
Female                                680
Name: location, dtype: int64

In [13]:
f1_all_feature_text = []
model_list = {}
for feature_text in (set(df_val.feature_text)):
  df_val2 = df_val[df_val.feature_text==feature_text]
  df_val2.reset_index(drop=True, inplace=True)
  df_val2['ftr_enc'] = df_val2.apply(lambda x: evaluateEncode(x.pn_historyNBME.split("#*#"), x.annotation_grp_cnt, x.location), axis=1)
  df_prep = pd.DataFrame(df_val2.apply(lambda x: prepareData_v1(x.ftr_enc, x.conf_df), axis=1))
  df_prep['annotation_grp_cnt'] = df_val2.annotation_grp_cnt
  df_prep.columns = ['data_prep1','annotation_grp_cnt']
  df_prep['XY'] = df_prep.apply(lambda x: getXY(x.data_prep1, x.annotation_grp_cnt), axis=1)
  x_train = []
  y_train = []
  for xy_1 in df_prep.XY:
    for x_1 in xy_1[0]:
      x_train.append(x_1)
    for y_1 in xy_1[1]:
      y_train.append(y_1)
  ######ML Model########
  model_dt = DecisionTreeClassifier()
  model_dt.fit(x_train, y_train)
  model_list[feature_text] = model_dt

  y_pred = model_dt.predict(x_train)
  
  curr_ptr = 0
  df_val2['loc_pred'] = df_val2.ftr_enc.apply(lambda x: predSlices(get_next_sequence_pred(x)))
  y_f1_true = df_val2.location.apply(getExpanded)
  y_f1_pred = df_val2.loc_pred.apply(lambda x: str(x).replace("[[","['").replace("]]","']").replace(",","").replace("] [",";")).apply(lambda x: getExpanded(ast.literal_eval(x)))

  f1_scores = []
  for i in range(len(y_f1_true)):
    f1_scores.append(getF1Score(y_f1_true[i], y_f1_pred[i]))
  f1_all_feature_text.append(sum(f1_scores) / len(f1_scores))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [14]:
sum(f1_all_feature_text) / len(f1_all_feature_text)

0.9070205229848898

**Predict**

In [15]:
#df_val2.loc_pred.apply(lambda x: str(x).replace("[[","['").replace("]]","']").replace(",","").replace("] [",";")).apply(lambda x: (ast.literal_eval(x))[0]).to_csv("submission.csv", index=False)

In [16]:
df_test['pn_historyNBME'] = df_test.pn_history.apply(tokenizeNBME)
df_test = pd.merge(df_test, ftr2,  left_on='feature_text', right_on='feature_text_grp_cnt').drop(columns=['feature_text_grp_cnt'])
df_test = pd.merge(df_test, ftr3[ftr3.columns[1:]],  left_on='feature_text', right_on='feature_text_conf').drop(columns=['feature_text_conf'])
df_test.reset_index(drop=True, inplace=True)
df_test['ftr_enc'] = df_test.apply(lambda x: evaluateEncode_v2(x.pn_historyNBME.split("#*#"), x.annotation_grp_cnt), axis=1)

In [17]:
df_test['loc_pred'] = df_test.apply(lambda x: predSlices(get_next_sequence_pred_v2(x.ftr_enc,model_list[x.feature_text].predict(get_x_train_v2(x.ftr_enc, x.conf_df, x.annotation_grp_cnt)))), axis=1)
df_test

,id,case_num_x,pn_num,feature_num,case_num_y,pn_history,case_num,feature_text,pn_historyNBME,annotation_grp_cnt,annotation_flt,annotation_vld,conf_df,ftr_enc,loc_pred
0,00016_000,0,16,0,0,HPI: 17yo M presents with palpitations. Patien...,0,Family-history-of-MI-OR-Family-history-of-myoc...,HPI#*#:#*##*# #*#17yo#*# #*#M#*# #*#presents#*...,"{'dad': 26, 'recent': 4, 'heart': 35, 'attcak'...","[dad with recent heart attcak, father: heart a...","[suffer, mi, father, acute, 1, attcak, in, the...",suffer mi father acute 1 attc...,"[[startOfSequence, 1, 0, 27], [heart, 96, 36, ...","[[696, 724]]"
1,00016_001,0,16,1,0,HPI: 17yo M presents with palpitations. Patien...,0,Family-history-of-thyroid-disorder,HPI#*#:#*##*# #*#17yo#*# #*#M#*# #*#presents#*...,"{'mom': 34, 'thyroid': 83, 'disease': 17, 'pro...","[mom with ""thyroid disease, mom has thyroid pr...","[in, diease, of, problem, condition, disorder,...",in diease of problem cond...,"[[startOfSequence, 1, 0, 22], [mom, 668, 282, ...","[[668, 693]]"
2,00016_002,0,16,2,0,HPI: 17yo M presents with palpitations. Patien...,0,Chest-pressure,HPI#*#:#*##*# #*#17yo#*# #*#M#*# #*#presents#*...,"{'chest': 68, 'pressure': 62, 'midsternal': 2,...","[chest pressure, pressure on her chest, chest ...","[center, sensation, in, the, discomfort, of, m...",center sensation in the discom...,"[[startOfSequence, 1, 0, 14], [chest, 129, 48,...","[[203, 217]]"
3,00016_003,0,16,3,0,HPI: 17yo M presents with palpitations. Patien...,0,Intermittent-symptoms,HPI#*#:#*##*# #*#17yo#*# #*#M#*# #*#presents#*...,"{'intermittent': 16, 'episodes': 65, 'episode'...","[intermittent episodes, episode, comes and goe...","[about, 1, 2, once, occured, in, random, the, ...",about 1 2 once occured in...,"[[startOfSequence, 1, 0, 38], [palpitations, 2...","[[70, 91], [176, 183]]"
4,00016_004,0,16,4,0,HPI: 17yo M presents with palpitations. Patien...,0,Lightheaded,HPI#*#:#*##*# #*#17yo#*# #*#M#*# #*#presents#*...,"{'felt': 6, 'going': 3, 'pass': 3, 'light': 14...","[felt as if he were going to pass out, light h...","[felt, lighheadeness, passing, that, was, ligh...",felt lighheadeness passing ...,"[[startOfSequence, 1, 0, 19], [felt, 222, 90, ...","[[222, 226], [241, 254]]"


In [18]:
df_test['location'] = df_test.loc_pred.apply(lambda x: str(x).replace("[[","['").replace("]]","']").replace(",","").replace("] [",";")).apply(lambda x: (ast.literal_eval(x))[0])

In [19]:
df_test[['id', 'location']].to_csv("submission.csv", index=False)